In [1]:
import random

# Global permission flag used by the setter
admin = False 

class BankAccount:
    # 2. CLASS VARIABLE: Shared by all instances to track every account created
    all_accounts = []

    def __init__(self, account_number, balance):
        # 2. INSTANCE ATTRIBUTES: Unique data for this specific object
        self.account_number = account_number
        
        # 2. ENCAPSULATION: Using double underscores to make balance 'private'
        self.__balance = balance
        
        # 2. AUTOMATIC REGISTRATION: Add this new object to our master list
        BankAccount.all_accounts.append(self)

    # 3. SILENT MODE: Optional parameters to control console feedback
    def deposit(self, amount, silent=False):
        if amount < 0:
            print("Only positive amounts can be deposited")
            return
        self.__balance += amount
        if not silent:
            print(f"Successfully deposited: ${amount}")

    def withdraw(self, amount, silent=False):
        # 4. BOOLEAN RETURNS: Returns True/False to allow conditional checks
        if amount < 0:
            print("Only positive amounts can be withdrawn")
            return False
        if self.__balance >= amount:
            self.__balance -= amount
            if not silent:
                print(f"Successfully withdrawn: ${amount}")
            return True
        else:
            print("Insufficient funds")
            return False

    # 3. PROPERTIES: The @property decorator creates a 'getter'
    @property
    def balance(self):
        return self.__balance

    # 5. SETTERS: The .setter controls updates and checks permissions
    @balance.setter
    def balance(self, amount):
        # 3 & 6. SETTER LIMITATIONS: Checking the global 'admin' flag
        if admin:
            self.__balance = amount
        else:
            print("You are not an admin")

    # 2. DUNDER METHODS: Controls how the object appears in print()
    def __str__(self):
        return f"Account Number: {self.account_number}\nBalance: ${self.__balance}"

    # 4. METHOD CHAINING: transfer calls withdraw and deposit
    def transfer(self, recipient, amount):
        # 4 & 6. THE EXECUTION TRAP: Calling withdraw() here actually runs it!
        if self.withdraw(amount, silent=True):
            recipient.deposit(amount, silent=True)
            print(f"Successfully transferred ${amount} to {recipient.account_number}")
        else:
            print("Insufficient funds")

1. The Blueprint and Automatic Registration
In this stage, we define how an account is born and how the class keeps track of every instance created.

Concepts:
Class-Level vs. Instance Attributes: all_accounts is a "Class Variable" shared by every account. self.account_number is an "Instance Variable" unique to each.

Automatic Registration: By adding BankAccount.all_accounts.append(self) inside __init__, we ensure no account is ever "lost" in memory.

Object Identity: Even if we create an account without assigning it a name (e.g., BankAccount("ID123", 100)), it survives because the class list holds a reference to it.

In [3]:
# 1. DEMO: Automatic account creation and registration
"""
class BankAccount:
    # 1. CLASS VARIABLE: Shared by all instances to track every account created
    all_accounts = []

    def __init__(self, account_number, balance):
        # 2. INSTANCE ATTRIBUTES: Unique data for this specific object
        self.account_number = account_number
        
        # 3. ENCAPSULATION: Using double underscores to make balance 'private'
        self.__balance = balance
        
        # 4. AUTOMATIC REGISTRATION: Add this new object to our master list
        BankAccount.all_accounts.append(self)
        
        print(f"--- System: Account {self.account_number} initialized and registered. ---")
"""

# Step-by-Step Execution Demo:
print("Step 1: Creating 3 accounts automatically...")
for i in range(3):
    BankAccount(f"Demo-{100 + i}", 500)

print(f"\nTotal accounts in registry: {len(BankAccount.all_accounts)}")

Step 1: Creating 3 accounts automatically...

Total accounts in registry: 6


2. Encapsulation and Pythonic Access
We want to protect the balance from being changed accidentally while making it easy to read.

Concepts:
Encapsulation (Private Variables): Using self.__balance (double underscore) "hides" the data from direct external access (e.g., acc.__balance will throw an error).

The @property Decorator: This turns a method into a "getter." You can access account.balance like a simple variable instead of calling a function like account.get_balance().

Dunder Methods (__str__): This "Double Underscore" method defines exactly what happens when you print(account).

In [6]:
"""
# Adding these methods to our class logic...
@property
def balance(self):
        '''Getter: Allows reading the private __balance.'''
        return self.__balance

def __str__(self):
        '''Dunder Method: Provides a user-friendly string representation.'''
        return f"Account Number: {self.account_number} | Balance: ${self.__balance}"
"""
        
# Demo of __str__ and @property
sample_acc = BankAccount.all_accounts[0]
print(f"Using @property: The balance is {sample_acc.balance}")
print(f"Using __str__:\n{sample_acc}")

Using @property: The balance is 500
Using __str__:
Account Number: Demo-100
Balance: $500


3. Method Design and the "Silent Mode" Pattern
When building methods that interact with each other, we need to control the feedback the user sees.

Concepts:
Boolean Returns: withdraw returns True or False. This allows us to use the method as a condition in if statements.

Silent Mode (Optional Parameters): Using silent=False as a default argument allows us to reuse the same logic for manual entries (which print) and automated transfers (which stay quiet).

In [7]:
"""
def deposit(self, amount, silent=False):
        if amount < 0:
            print("Error: Only positive amounts can be deposited")
            return
        self.__balance += amount
        if not silent:
            print(f"Success: Deposited ${amount} to {self.account_number}")

def withdraw(self, amount, silent=False):
    if amount < 0:
        print("Error: Only positive amounts can be withdrawn")
        return False
    if self.__balance >= amount:
        self.__balance -= amount
        if not silent:
            print(f"Success: Withdrawn ${amount} from {self.account_number}")
        return True
    else:
        print(f"Error: Insufficient funds in {self.account_number}")
        return False
"""
# Demo of deposit and withdraw with silent mode

'\ndef deposit(self, amount, silent=False):\n        if amount < 0:\n            print("Error: Only positive amounts can be deposited")\n            return\n        self.__balance += amount\n        if not silent:\n            print(f"Success: Deposited ${amount} to {self.account_number}")\n\ndef withdraw(self, amount, silent=False):\n    if amount < 0:\n        print("Error: Only positive amounts can be withdrawn")\n        return False\n    if self.__balance >= amount:\n        self.__balance -= amount\n        if not silent:\n            print(f"Success: Withdrawn ${amount} from {self.account_number}")\n        return True\n    else:\n        print(f"Error: Insufficient funds in {self.account_number}")\n        return False\n'

4. Logic Interaction and "The Execution Trap"
This is where the class becomes functional, but it requires careful logic to avoid bugs.

Concepts:
The Execution Trap: In the line if self.withdraw(amount, silent=True):, Python doesn't just "check" if the withdrawal is possible—it runs the whole function, changing the balance immediately.

The "Double Deduction" Bug: Because withdraw already subtracts the money, we must not subtract it again inside the transfer method.

Method Chaining: transfer simply orchestrates existing building blocks (withdraw and deposit).

In [10]:
"""
def transfer(self, recipient, amount):
        print(f"\n--- Initiating Transfer: ${amount} from {self.account_number} to {recipient.account_number} ---")
        
        # TRAP AVOIDANCE: This 'if' executes the withdrawal immediately.
        if self.withdraw(amount, silent=True):
            # If withdrawal worked, deposit to the recipient silently.
            recipient.deposit(amount, silent=True)
            print("Transfer Complete!")
        else:
            print("Transfer Failed!")
"""
            
print(f"Account A Balance Before Transfer: ${BankAccount.all_accounts[0].balance}")
print(f"Account B Balance Before Transfer: ${BankAccount.all_accounts[1].balance}")

# Execution Demo
acc_a = BankAccount.all_accounts[0]
acc_b = BankAccount.all_accounts[1]
acc_a.transfer(acc_b, 100)

print(f"Account A Balance After Transfer: ${acc_a.balance}")
print(f"Account B Balance After Transfer: ${acc_b.balance}")

Account A Balance Before Transfer: $300
Account B Balance Before Transfer: $700
Successfully transferred $100 to Demo-101
Account A Balance After Transfer: $200
Account B Balance After Transfer: $800


5. Security and Setter Limitations
Finally, we implement administrative overrides using properties.

Concepts:
The .setter Decorator: This controls how a value is updated. We use it to check the global admin flag before allowing a manual balance override.

The Setter Argument Rule: A setter can only take exactly one value argument (the new value). You cannot write def balance(self, value, admin):, which is why we check a global or external permission flag.

In [13]:
"""
@balance.setter
def balance(self, amount):
    if admin:
        print(f"Admin override: Setting {self.account_number} balance to ${amount}")
        self.__balance = amount
    else:
        print(f"Security Alert: Non-admin attempted to override balance on {self.account_number}")
"""
        
# Demo of Setter Security
print("\n--- Testing Balance Setter Security ---")
print(f"Current Balance: ${sample_acc.balance}")

admin = False
sample_acc.balance = 99999  # Should fail

print(f"Balance after non-admin attempt: ${sample_acc.balance}")

admin = True
sample_acc.balance = 99999  # Should succeed

print(f"Balance after admin override: ${sample_acc.balance}")


--- Testing Balance Setter Security ---
Current Balance: $99999
You are not an admin
Balance after non-admin attempt: $99999
Balance after admin override: $99999


6. Scaling to 100+ Accounts
By combining all the concepts above, we can initialize and manage a massive amount of data with just a few lines of code

In [16]:
# Resetting for a clean run
BankAccount.all_accounts = []
admin = False

print("Generating 100 random accounts...")
for i in range(100):
    acc_id = f"Account-{1000 + i}"
    start_bal = random.randint(100, 5000)
    # These exist in BankAccount.all_accounts immediately
    BankAccount(acc_id, start_bal)

print(f"\nCreated {len(BankAccount.all_accounts)} accounts.")
print(f"Example Account Status: {BankAccount.all_accounts[50]}")

Generating 100 random accounts...

Created 100 accounts.
Example Account Status: Account Number: Account-1050
Balance: $1416


Test Code

In [ ]:
import random


admin = False
try:
  if input("Enter password:" ) == "admin123":
    admin = True
  else:
    admin = False
except KeyboardInterrupt:
  print("Invalid password")



class BankAccount:

    all_accounts = []

    def __init__(self, account_number, balance):
        self.account_number = account_number
        self.__balance = balance
        BankAccount.all_accounts.append(self)

    def deposit(self, amount, silent=False):
        if amount < 0:
          print("Only positive amounts can be deposited")
          return
        self.__balance += amount
        if not silent:
          print(f"Successfully deposited: ${amount}")

    def withdraw(self, amount, silent=False):
        if amount < 0:
          print("Only positive amounts can be withdrawn")
          return False
        if self.__balance >= amount:
            self.__balance -= amount
            if not silent:
              print(f"Successfully withdrawn: ${amount}")
            return True
        else:
            print("Insufficient funds")
            return False

    @property
    def balance(self):
        return self.__balance

    @balance.setter
    def balance(self, amount):
        if admin:
            self.__balance = amount
        else:
            print("You are not an admin")

    def __str__(self):
        return f"Account Number: {self.account_number}\nBalance: ${self.__balance}"

    def transfer(self, recipient, amount):
      if self.withdraw(amount, silent=True):
        recipient.deposit(amount, silent=True)
        print(f"Successfully transferred ${amount} to {recipient.account_number}")
      else:
        print("Insufficient funds")


for i in range(100):
  acc_id = f"Account-{1000 + i}"
  start_bal = random.randint(100, 5000)
  BankAccount(acc_id, start_bal)




819
1336
Successfully transferred $500 to Account-1001
Account Number: Account-1000
Balance: $319
Account Number: Account-1001
Balance: $1836
Account Number: Account-1000
Balance: $319
Account Number: Account-1001
Balance: $1836
Account Number: Account-1000
Balance: $819
Account Number: Account-1001
Balance: $2336
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully deposited: $500
Successfully depo